# Neuron Model for Thalamocortical cells

In [1]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt

In [27]:
bm.exp(2)

Array(7.389056, dtype=float32, weak_type=True)

Here we are trying to reproduce the thalamocortical model in the following papers:  

- Li G, Henriquez CS, Fröhlich F (2017) Unified thalamic model generates multiple distinct oscillations with state-dependent entrainment by stimulation. PLoS Comput Biol 13(10): e1005797. https://doi.org/10.1371/journal.pcbi.1005797  
- Gu Q L, Lam N H, Wimmer R D, et al. Computational circuit mechanisms underlying thalamic control of attention[J]. bioRxiv, 2021.

## The model

Following previous “point” models of thalamic cells, all single cell models in the thalamic network contained one single compartment and multiple ionic currents described by the Hodgkin-Huxley formulism. The current balance equation was given by:  

$$  
C_m \frac{d V}{d t}=-g_L\left(V-E_L\right)-g_{K L}\left(V-E_{K L}\right)-\sum I^{i n t}-10^{-3} \sum \frac{I^{s n}}{A}+10^{-3} \frac{I_{a p p}}{A}  
$$  


where $Cm = 1μF/cm^2$ is the membrane capacitance for all four types of neurons, $g_L$ is the leakage conductance (nominal value: $gL = 0.01 mS/cm^2$ for all four types of cells) and $g_{KL}$ is the potassium leak conductance modulated by both ACh and NE. $E_L$ is the leakage reversal potential ($E_L$ = −70 mV for both HTC cells), and $E_{KL}$ is the reversal potential for the potassium leak current ($E_{KL}$ = −90 mV for all neurons). $I_{int}$ and $I_{syn}$ are the intrinsic ionic currents (in $μA/cm^2$) and synaptic currents (in $nA$) respectively and $I_{app}$ is the externally applied current injection (in $nA$). The following total membrane area (A) was used to normalize the synaptic and externally applied currents in Eq: HTC cells: 2.9×10−4 $cm^2$.

HTC cells contained the following six active ionic currents:  

- a spike generating fast sodium current (INa),  ``bp.dyn.INa_Ba2002``  
- a delayed rectifier potassium current (IDR), ``bp.dyn.IKDR_Ba2002``  
- a hyperpolarization-activated cation current (IH), ``bp.dyn.Ih_HM1992``  
- a high-threshold L-type Ca2+ current (ICa/L), ``bp.dyn.ICaL_IS2008``  
- a Ca2+- dependent potassium current (IAHP), ``bp.dyn.IAHP_De1994``  
- a Ca2+- activated nonselective cation current (ICAN). ``bp.dyn.ICaN_IS2008``  

In addition, both TC cells included  
- a regular low-threshold T-type Ca2+ current (ICa/T), ``bp.dyn.ICaT_HM1992``  
- and a high-threshold T-type Ca2+ current (ICa/HT); ``bp.dyn.ICaHT_HM1992``  

To obtain the high-threshold T-type current ICa/HT, both the activation and inactivation of the ICa/T current was shifted by 28 mV, similar to a previous TC modeling study. 

## Task 1  

1. implement the above HTC neuron model  
2. give the stimulus, and reproduce the rebound bursting firing pattern of the HTC cell

In [2]:
class HTC(bp.dyn.CondNeuGroupLTC):
    def __init__(self,size,gKL=0.01, V_initializer=bp.init.OneInit(-65.)):
        super().__init__(size,A=2.9e-4,V_initializer=V_initializer,V_th=20.)
        self.IL = bp.dyn.IL(size,g_max=0.01,E=-70.)
        self.INa = bp.dyn.INa_Ba2002(size,V_sh=-30)
        self.Ih = bp.dyn.Ih_HM1992(size,g_max=0.01,E=-43)

        self.Ca = bp.dyn.CalciumDetailed(size,C_rest=5e-5,tau=10.,d=0.5)
        self.Ca.add_elem(bp.dyn.ICaL_IS2008(size,g_max=0.5))
        self.Ca.add_elem(bp.dyn.ICaN_IS2008(size,g_max=0.5))
        self.Ca.add_elem(bp.dyn.ICaT_HM1992(size,g_max=2.1))
        self.Ca.add_elem(bp.dyn.ICaHT_HM1992(size,g_max=3.0))

        self.K = bp.dyn.PotassiumFixed(size,E=-90.)
        self.K.add_elem(bp.dyn.IKDR_Ba2002v2(size,V_sh=-30.,phi=0.25))
        self.K.add_elem(bp.dyn.IK_Leak(size,g_max=gKL))

        self.KCa = bp.dyn.MixIons(self.K,self.Ca)
        self.KCa.add_elem(bp.dyn.IAHP_De1994v2(size))


In [3]:
htc = HTC(1)
runner = bp.DSRunner(htc,monitors={'v':htc.V})
I = -30/1e3/2.9e-4*1e-3 # input current = -30pA
inputs = np.ones(20000)*I
runner.run(inputs=inputs)
bp.visualize.line_plot(runner.mon.ts,runner.mon['v'],legend='v',show=True)

  0%|          | 0/20000 [00:00<?, ?it/s]

<Figure size 432x288 with 1 Axes>

## Task 2  


The next task is to implement the ``TC neuron model`` used in the following paper:  

- Gu Q L, Lam N H, Wimmer R D, et al. Computational circuit mechanisms underlying thalamic control of attention[J]. bioRxiv, 2021.  

![](figs/tc-fig1.png)  
![](figs/tc-fig2.png)  
![](figs/tc-fig3.png)  


In [64]:
class CaLT(bp.dyn.IonChannel):
    master_type = bp.dyn.HHTypedNeuron
    def __init__(self,size,g_max=1.4,E=120.,T=36.0,method='exp_auto'):
        super().__init__(size)
        self.g_max = g_max
        self.E = E
        self.T = T
        self.m = bm.Variable(bm.zeros(size))
        self.h = bm.Variable(bm.zeros(size))
        self.d = bm.Variable(bm.zeros(size))

        self.integral = bp.odeint(bp.JointEq(self.dm,self.dh,self.dd),method=method)
    
    def dm(self,m,t,V):
        m_Inf = 1/(1+bm.exp((V+65.)/-7.8))
        phi_m = 5**((self.T-24.)/10.)
        tau_Ca = m_Inf*(1+bm.exp((V+30.8)/-13.5))/phi_m
        return (m_Inf-m)/tau_Ca

    def dh(self,h,t,d,V):
        phi_h = 3**((self.T-24.)/10.)
        a = bm.exp((V+162.3)/-17.8)*phi_h
        k = bm.exp((V+85.5)/2)/(6.3**0.5)-0.5
        b = a*k
        return a*(1-h-d)-b*h

    def dd(self,d,t,h,V):
        phi_h = 3**((self.T-24.)/10.)
        k = bm.exp((V+85.5)/2)/(6.3**0.5)-0.5
        a = (1+bm.exp(V+39.4)/30)/(240.*(1+k))*phi_h
        b = a*k
        return b*(1-h-d)-a*d
    
    def reset_state(self,V,batch_or_ormode=None,*args,**kwargs):
        self.m = bp.init.variable_(bm.zeros,self.num,batch_or_mode)
        self.h = bp.init.variable_(bm.zeros,self.num,batch_or_mode)
        self.d = bp.init.variable_(bm.zeros,self.num,batch_or_mode)

    def update(self,V,*args,**kwargs):
        t = bp.share.load('t')
        dt = bp.share.load('dt')
        self.m.value,self.h.value,self.d.value = self.integral(self.m,self.h,self.d,t,V,dt=dt)
    
    def current(self,V,*args,**kwargs):
        return self.g_max * self.m**3 * self.h * (V-self.E)

class CaH(bp.dyn.IonChannel):
    master_type = bp.dyn.Calcium
    def __init__(self,size,g_max=0.05,E=-43.,T=36.0,k1=25,k2=4e-4,k3=0.1,k4=1e-3,method='exp_auto'):
        super().__init__(size)
        self.g_max = g_max
        self.E = E
        self.T = T
        self.k1 = k1
        self.k2 = k2
        self.k3 = k3
        self.k4 = k4
        self.O1 = bm.Variable(bm.zeros(size))
        self.O2 = bm.Variable(bm.zeros(size))
        self.p = bm.Variable(bm.zeros(size))

        self.integral = bp.odeint(bp.JointEq(self.dO1,self.dO2,self.dp),method=method)

    def dO1(self,O1,t,O2,p,V):
        h_Inf = 1/(1+bm.exp((V+75)/5.5))
        phi_m = 3**((self.T-24)/10)
        tau_h = 1/phi_m*(20+1000/(bm.exp((V+71.5)/14.2)+bm.exp((V+89)/11.6)))
        a = h_Inf/tau_h
        b = (1-h_Inf)/tau_h
        return a*(1-O1-O2)-b*O1+self.k4*O2-self.k3*self.p*O1

    def dO2(self,O2,t,O1,p):
        return -1e-3*O2+0.1*p*O1
    
    def dp(self,p,t,C):
        return -1*self.k2*p+self.k1*C*(1-p)

    def reset_state(self,V,C,batch_or_ormode=None,*args,**kwargs):
        self.O1 = bp.init.variable_(bm.zeros,self.num,batch_or_mode)
        self.O2 = bp.init.variable_(bm.zeros,self.num,batch_or_mode)
        self.p = bp.init.variable_(bm.zeros,self.num,batch_or_mode)
    
    def update(self,V,C,*args,**kwargs):
        t = bp.share.load('t')
        dt = bp.share.load('dt')
        self.O1.value,self.O2.value,self.p.value = self.integral(self.O1,self.O2,self.p,t,V,C,dt=dt)

    def current(self,V,*args,**kwargs):
        return self.g_max * (self.O1+2*self.O2) * (V-self.E)

class TC(bp.dyn.CondNeuGroupLTC):
    def __init__(self,size,V_initializer=bp.init.OneInit(-65.)):
        super().__init__(size,V_initializer=V_initializer)
        self.IL = bp.dyn.IL(size,g_max=0.05,E=-90.)
        self.INa = bp.dyn.INa_Ba2002(size,g_max=30.,V_sh=-55.)
        self.IK = bp.dyn.IK_TM1991(size,E=-95.,g_max=2.)
        self.ICa = CaLT(size)
        #self.Ca = bp.dyn.CalciumDetailed(size)
        #self.Ca.add_elem(CaH(size))


In [65]:
tc = TC(1)
runner = bp.DSRunner(tc, monitors={'v': tc.V,'ICa_m':tc.ICa.m,'ICa_h':tc.ICa.h,'ICa_d':tc.ICa.d})
#runner = bp.DSRunner(tc, monitors={'v': tc.V})
I = 10
inputs = np.ones(2000)*I
runner.run(inputs=inputs)
bp.visualize.line_plot(runner.mon.ts, runner.mon['v'], legend='v', show=True)
bp.visualize.line_plot(runner.mon.ts, runner.mon['ICa_m'], legend='ICa_m', show=True)
bp.visualize.line_plot(runner.mon.ts, runner.mon['ICa_h'], legend='ICa_h', show=True)
bp.visualize.line_plot(runner.mon.ts, runner.mon['ICa_d'], legend='ICa_d', show=True)
m_Inf = 1/(1+bm.exp((runner.mon['v']+65.)/-7.8))
phi_m = 5**((36-24.)/10.)
phi_h = 3**((36-24.)/10.)
tau_Ca = m_Inf*(1+bm.exp((runner.mon['v']+30.8)/-13.5))/phi_m

bp.visualize.line_plot(runner.mon.ts, 1.4*runner.mon['ICa_m']**3*runner.mon['ICa_h']*(runner.mon['v']-120), legend='ICa', show=True)

  0%|          | 0/2000 [00:00<?, ?it/s]

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>